# Biconvex Lens Parabola

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm


class Optics(tlm.Module):
    def __init__(self):
        super().__init__()
        self.shape = tlm.Parabola(height=15., a=nn.Parameter(torch.tensor(0.005)))

        self.lens = tlm.SymmetricLens(self.shape, (1.0, 1.49), outer_thickness=0.5)

        self.optics = tlm.OpticalSequence(
            tlm.PointSourceAtInfinity(beam_diameter=25),
            tlm.Gap(10.),
            
            self.lens,
            
            tlm.Gap(45.0),
            tlm.FocalPoint(),
        )

    def forward(self, inputs, sampling):
        return self.optics(inputs, sampling)

optics = Optics()

tlm.render_plt(optics)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"rays": 10},
    num_iter = 100
)

tlm.render_plt(optics)

In [ ]:
from IPython.display import display
import build123d as bd

part = tlm.lens_to_part(optics.lens)
display(part)

print("Outer thickness:", optics.lens.outer_thickness().item())
print("Inner thickness:", optics.lens.inner_thickness().item())

# bd.export_step(part, "lens.step")